In [ ]:
import pandas as pd
import re
import pandas_datareader as dr
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


%store -r

In [ ]:
# %run PrepareData.ipynb

In [ ]:
# tsla_tweets_df.head()

In [ ]:
# The purpose of this function is to prepare text for lexicon by removing special characters and symbols that are irrelavant for the tweet classification
def cleanTweet(text):
    # Remove '#' hash tag     
    text = re.sub('#', '', text) 
    #Remove @mentions 
    text = re.sub('@[A-Za-z0–9]+', '', text) 
    # Remove RT
    tweet = re.sub('RT[\s]+', '', text) 
    # Remove hyperlink
    tweet = re.sub('https?:\/\/\S+', '', text)  
    return text
def getSentiments(tweet):
    sentiments = analyser.polarity_scores(cleanTweet(tweet))    
    return sentiments["neg"],sentiments["neu"],sentiments["pos"],sentiments["compound"]


In [ ]:
analyser = SentimentIntensityAnalyzer()
tsla_tweets_df["polarityScore"] = tsla_tweets_df["body"].apply(lambda tweet : analyser.polarity_scores(cleanTweet(tweet))["compound"] )
# tsla_tweets_df[["neg","neu","pos","compound"]] = tsla_tweets_df["body"].apply(lambda tweet: getSentiments(tweet))
tsla_tweets_df["sentiment"] = tsla_tweets_df["polarityScore"].apply(lambda ps : 1 if(ps >=0.05) else 0 )
# display(tsla_tweets_df.head())
# display(tsla_tweets_df.tail())

In [ ]:
tsla_sentiments_df = tsla_tweets_df[["post_date","polarityScore"]].copy()
tsla_sentiments_df['Date'] = pd.to_datetime(tsla_tweets_df['post_date'].dt.date, utc=False)
tsla_sentiments_df = tsla_sentiments_df.groupby('Date').mean()

In [ ]:
%store tsla_sentiments_df